In [1]:
import pandas as pd
import numpy as np
from neighborhoods import madrid_districts_dict
import re
from shapely.geometry import Point, Polygon

In [2]:
df = pd.read_json('../data/final_scraped_all_22Jul.json')

In [3]:
df.head()

,Title,Price,Url,Bedrooms,Area,Full description,Short description,Features
0,"Piso en calle de Toledo, Acacias, Madrid",209.995€,/inmueble/105157270/,2 hab.,50 m²,! ATENCION INVERSORES VIVIENDA CON 8% DE RENTA...,! ATENCION INVERSORES VIVIENDA CON 8% DE RENTA...,"[50 m² construidos, 2 habitaciones, 1 baño, Te..."
1,"Piso en calle de Carlos Arniches, Lavapiés-Emb...",209.995€,/inmueble/105129078/,2 hab.,50 m²,! ATENCION INVERSORES VIVIENDA CON 8% DE RENTA...,! ATENCION INVERSORES VIVIENDA CON 8% DE RENTA...,"[50 m² construidos, 2 habitaciones, 1 baño, Te..."
2,"Piso en Villaverde Alto, Madrid",180.000€,/inmueble/105406757/,1 hab.,50 m²,¡Descubre tu nuevo hogar en Villaverde Alto! E...,¡Descubre tu nuevo hogar en Villaverde Alto! E...,"[50 m² construidos, 1 habitación, 1 baño, Plaz..."
3,"Piso en calle de la Ballesta, Malasaña-Univers...",290.000€,/inmueble/103865983/,1 hab.,50 m²,MAGNÍFICO PISO EN VENTA EN LA CONFLUENCIA DE M...,MAGNÍFICO PISO EN VENTA EN LA CONFLUENCIA DE M...,"[50 m² construidos, 45 m² útiles, 1 habitación..."
4,"Piso en calle de Samaniego, Rejas, Madrid",176.000€,/inmueble/104558678/,1 hab.,50 m²,GPHOME VENDE MAGNIFICO APARTAMENTO TURISTICO E...,GPHOME VENDE MAGNIFICO APARTAMENTO TURISTICO E...,"[50 m² construidos, 1 habitación, 1 baño, Plaz..."


In [4]:
df = df[['Price', 'Url', 'Title', 'Full description', 'Bedrooms', 'Features', 'Area']]

In [5]:
df['Price'] = df['Price'].str.replace('.', '').str.replace('€', '').apply(lambda x: int(x))
df['price'] = df['Price'].copy()

/var/folders/zm/z5q67jgd0vv23h_ypgdpl9ph0000gn/T/ipykernel_99089/2632506402.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Price'] = df['Price'].str.replace('.', '').str.replace('€', '').apply(lambda x: int(x))


In [6]:
df_area_values = []
not_found = []
for index, item in enumerate(df['Features'].values):
    found = False
    for value in item:
        if 'construidos' in value:
            df_area_values.append(value)
            found = True
            break
    if not found:
        if 'm²' in df['Area'].iloc[index]:
            df_area_values.append(df['Area'].iloc[index])
        else:
           df_area_values.append(df['Bedrooms'].iloc[index])

df['m2'] = df_area_values

df.iloc[not_found]

df['m2'] = df['m2'].str.replace('.', '')
df['m2'] = df['m2'].apply(lambda x: float(x.split()[0]))

/var/folders/zm/z5q67jgd0vv23h_ypgdpl9ph0000gn/T/ipykernel_99089/2424817073.py:20: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['m2'] = df['m2'].str.replace('.', '')


In [7]:
df['Features'] = df['Features'].apply(lambda x: ', '.join(x))

In [8]:
df = df.drop_duplicates()

In [9]:
df_rooms_values = []

for index, value in enumerate(df['Features'].values):
    if 'habitaciones' in value:
        pattern = r'\b(\w+)\s+habitaciones\b'
        match = re.search(pattern, value)
        word_before = match.groups()
        df_rooms_values.append(f'{word_before[0]}')
    elif 'habitación' in value:
        pattern = r'\b(\w+)\s+habitación\b'
        match = re.search(pattern, value)
        word_before = match.groups()
        df_rooms_values.append(f'{word_before[0]}')
    elif 'hab.' in df['Bedrooms'].iloc[index]:
        df_rooms_values.append(df['Bedrooms'].iloc[index].split(' ')[0])
    else:
        df_rooms_values.append('not found')
    
df['rooms'] = df_rooms_values
df = df[df['rooms']!='not found']
df['rooms'] = df['rooms'].replace('Sin', '0').apply(lambda x: int(x))

In [10]:
df_house_type_2 = []

for index, item in enumerate(df['Features'].values):
    found = False
    if 'exterior' in item.lower() and 'interior' in item.lower():
        value = item.lower()
        pattern_ext = r'\b(\w+)\s+exterior\b'
        match_ext = re.search(pattern_ext, value)
        word_before_ext = match_ext.groups()
        pattern_int = r'\b(\w+)\s+interior\b'
        match_int = re.search(pattern_int, value)
        word_before_int = match_int.groups()
        if word_before_int[0] == 'solo':
            df_house_type_2.append('exterior')
        if word_before_ext[0] == 'acceso':
            df_house_type_2.append('interior')
        found = True
    elif 'exterior' in item.lower():
        df_house_type_2.append('exterior')
        found = True
    elif 'interior' in value.lower():
        df_house_type_2.append('interior')
        found = True
    else:
        df_house_type_2.append('not found')

df['house_type_2'] = df_house_type_2


In [11]:
has_garage = []

for index, item in enumerate(df['Features'].values):
    found = False
    value = item.lower()
    if 'garage' in value:
        pattern = r'\b(\w+)\s+garage\b'
        match = re.search(pattern, value)
        word_before = match.groups()
        if word_before_int[0] == 'sin':
            has_garage.append(0)
        else:
            has_garage.append(1)
    elif 'garaje' in value:
        pattern = r'\b(\w+)\s+garaje\b'
        match = re.search(pattern, value)
        word_before = match.groups()
        if word_before_int[0] == 'sin':
            has_garage.append(0)
        else:
            has_garage.append(1)
    else:
        has_garage.append(0)

df['garage'] = has_garage

In [12]:
df.head()

,Price,Url,Title,Full description,Bedrooms,Features,Area,price,m2,rooms,house_type_2,garage
0,209995,/inmueble/105157270/,"Piso en calle de Toledo, Acacias, Madrid",! ATENCION INVERSORES VIVIENDA CON 8% DE RENTA...,2 hab.,"50 m² construidos, 2 habitaciones, 1 baño, Ter...",50 m²,209995,50.0,2,interior,0
1,209995,/inmueble/105129078/,"Piso en calle de Carlos Arniches, Lavapiés-Emb...",! ATENCION INVERSORES VIVIENDA CON 8% DE RENTA...,2 hab.,"50 m² construidos, 2 habitaciones, 1 baño, Ter...",50 m²,209995,50.0,2,interior,0
2,180000,/inmueble/105406757/,"Piso en Villaverde Alto, Madrid",¡Descubre tu nuevo hogar en Villaverde Alto! E...,1 hab.,"50 m² construidos, 1 habitación, 1 baño, Plaza...",50 m²,180000,50.0,1,exterior,1
3,290000,/inmueble/103865983/,"Piso en calle de la Ballesta, Malasaña-Univers...",MAGNÍFICO PISO EN VENTA EN LA CONFLUENCIA DE M...,1 hab.,"50 m² construidos, 45 m² útiles, 1 habitación,...",50 m²,290000,50.0,1,interior,0
4,176000,/inmueble/104558678/,"Piso en calle de Samaniego, Rejas, Madrid",GPHOME VENDE MAGNIFICO APARTAMENTO TURISTICO E...,1 hab.,"50 m² construidos, 1 habitación, 1 baño, Plaza...",50 m²,176000,50.0,1,exterior,1


In [13]:
has_elevator = []

for index, item in enumerate(df['Features'].values):
    found = False
    value = item.lower()
    if 'ascensor' in value:
        pattern = r'\b(\w+)\s+ascensor\b'
        match = re.search(pattern, value)
        word_before = match.groups()
        if word_before_int[0] == 'sin':
            has_elevator.append(0)
        else:
            has_elevator.append(1)
    else:
        has_elevator.append(0)

df['elevator'] = has_elevator

In [14]:
df['house_type'] = df['Title'].apply(lambda x: x.split(' en ')[0])

df['house_type'] = df['house_type'].replace('Chalet adosado', 'Chalet').replace('Chalet pareado', 'Chalet')

df['house_type'] = df['house_type'].replace('Casa o chalet independiente', 'Casa')

In [15]:
floor_num = []
not_found = []

for index, item in enumerate(df['Features'].values):
    found = False
    item = item.lower()
    item = item.split(', ')
    for value in item:
        if 'planta' in value and 'entreplanta' not in value:
            value = value.replace(' plantas', '').replace(' planta', '').replace('planta ', '').replace(' exterior', '').replace(' interior', '').replace('ª', '')
            floor_num.append(value)
            found = True
            break
    if not found:
        not_found.append(index)
        floor_num.append(0) # floor not specified

df['floor'] = floor_num
df['floor'] = df['floor'].apply(lambda x: int(x))

In [16]:
built_year = []

for index, item in enumerate(df['Features'].values):
    found = False
    value = item.lower()
    if 'construido en' in value:
        pattern = r'construido en (\d{4})'
        match = re.search(pattern, value)
        year = int(match.group(1))
        built_year.append(year)
    else:
        built_year.append(None)

df['built_year'] = built_year

In [17]:
bathrooms = []

for index, item in enumerate(df['Features'].values):
    found = False
    value = item.lower()
    if 'baño' in value or 'baños' in value:
        if 'sin baños' in value:
            bathrooms.append(0)
        else:
            value = value.replace('baños', 'baño')
            pattern = r'(\d+) baño'
            match = re.search(pattern, value)
            bathroom = int(match.group(1))
            bathrooms.append(bathroom)
    else:
        bathrooms.append(0)

df['bathrooms'] = bathrooms

In [18]:
needs_renovation = []

for index, item in enumerate(df['Features'].values):
    found = False
    value = item.lower()
    if 'segunda mano/para reformar' in value:
        needs_renovation.append(1)
    else:
        needs_renovation.append(0)

df['needs_renovation'] = needs_renovation

In [19]:
has_ac = []

for index, item in enumerate(df['Features'].values):
    found = False
    value = item.lower()
    if 'aire acondicionado' in value:
        if 'sin aire acondicionado' in value:
            has_ac.append(0)
        else:
            has_ac.append(1)
    else:
        has_ac.append(0)

df['has_ac'] = has_ac

In [20]:
df.head()

,Price,Url,Title,Full description,Bedrooms,Features,Area,price,m2,rooms,house_type_2,garage,elevator,house_type,floor,built_year,bathrooms,needs_renovation,has_ac
0,209995,/inmueble/105157270/,"Piso en calle de Toledo, Acacias, Madrid",! ATENCION INVERSORES VIVIENDA CON 8% DE RENTA...,2 hab.,"50 m² construidos, 2 habitaciones, 1 baño, Ter...",50 m²,209995,50.0,2,interior,0,1,Piso,0,NaN,1,0,0
1,209995,/inmueble/105129078/,"Piso en calle de Carlos Arniches, Lavapiés-Emb...",! ATENCION INVERSORES VIVIENDA CON 8% DE RENTA...,2 hab.,"50 m² construidos, 2 habitaciones, 1 baño, Ter...",50 m²,209995,50.0,2,interior,0,1,Piso,0,NaN,1,0,0
2,180000,/inmueble/105406757/,"Piso en Villaverde Alto, Madrid",¡Descubre tu nuevo hogar en Villaverde Alto! E...,1 hab.,"50 m² construidos, 1 habitación, 1 baño, Plaza...",50 m²,180000,50.0,1,exterior,1,1,Piso,7,2007.0,1,0,1
3,290000,/inmueble/103865983/,"Piso en calle de la Ballesta, Malasaña-Univers...",MAGNÍFICO PISO EN VENTA EN LA CONFLUENCIA DE M...,1 hab.,"50 m² construidos, 45 m² útiles, 1 habitación,...",50 m²,290000,50.0,1,interior,0,1,Piso,1,1880.0,1,0,1
4,176000,/inmueble/104558678/,"Piso en calle de Samaniego, Rejas, Madrid",GPHOME VENDE MAGNIFICO APARTAMENTO TURISTICO E...,1 hab.,"50 m² construidos, 1 habitación, 1 baño, Plaza...",50 m²,176000,50.0,1,exterior,1,1,Piso,3,2006.0,1,0,0


In [21]:
has_pool = []

for index, item in enumerate(df['Features'].values):
    found = False
    value = item.lower()
    if 'piscina' in value:
        if 'sin piscina' in value:
            has_pool.append(0)
        else:
            has_pool.append(1)
    else:
        has_pool.append(0)

df['has_pool'] = has_pool

In [22]:
has_terrace = []

for index, item in enumerate(df['Features'].values):
    found = False
    value = item.lower()
    if 'terraza' in value:
        if 'sin terraza' in value:
            has_terrace.append(0)
        else:
            has_terrace.append(1)
    else:
        has_terrace.append(0)

df['has_terrace'] = has_terrace

In [23]:
has_orientation_east = []

for index, item in enumerate(df['Features'].values):
    value = item.lower()
    value = value.replace('norte, ', '').replace('sur, ', '').replace('oeste, ', '')
    if 'orientación este' in value:
        has_orientation_east.append(1)
    else:
        has_orientation_east.append(0)

df['has_orientation_east'] = has_orientation_east

In [24]:
has_orientation_north = []

for index, item in enumerate(df['Features'].values):
    value = item.lower()
    value = value.replace('este, ', '').replace('sur, ', '').replace('oeste, ', '')
    if 'orientación norte' in value:
        has_orientation_north.append(1)
    else:
        has_orientation_north.append(0)

df['has_orientation_north'] = has_orientation_north

In [25]:
has_orientation_west = []

for index, item in enumerate(df['Features'].values):
    value = item.lower()
    value = value.replace('este, ', '').replace('sur, ', '').replace('norte, ', '')
    if 'orientación oeste' in value:
        has_orientation_west.append(1)
    else:
        has_orientation_west.append(0)

df['has_orientation_west'] = has_orientation_west

In [26]:
has_orientation_south = []

for index, item in enumerate(df['Features'].values):
    value = item.lower()
    value = value.replace('este, ', '').replace('oeste, ', '').replace('norte, ', '')
    if 'orientación sur' in value:
        has_orientation_south.append(1)
    else:
        has_orientation_south.append(0)

df['has_orientation_south'] = has_orientation_south

In [27]:
madrid_neighborhoods = []

for key in madrid_districts_dict:
    madrid_neighborhoods.extend(madrid_districts_dict[key])

In [28]:
df['neighbourhood'] = df['Title'].apply(lambda x: x.split(', ')[-2]).apply(lambda x: x.split(' en ')[-1])

In [29]:
item_conversion_dict = {'Águilas': 'Las Águilas',
 'Palos de la Frontera': 'Palos de Moguer',
 'Malasaña-Universidad': 'Universidad',
 'Valdebebas - Valdefuentes': 'Valdefuentes',
 'Montecarmelo': 'El Goloso',
 'San Cristóbal': 'San Cristóbal de los Ángeles',
 'Delicias': 'Las Delicias',
 'Sanchinarro': 'Valdefuentes',
 'Acacias': 'Las Acacias',
 'Ventilla-Almenara': 'Almenara',
 'Las Tablas': 'Valverde',
 '12 de Octubre-Orcasur': 'Orcasur',
 'Ensanche de Vallecas - La Gavia': 'Casco Histórico de Vallecas',
 'Campo de las Naciones-Corralejos': 'Corralejos',
 'Palomeras sureste': 'Palomeras Sureste',
 'Conde Orgaz-Piovera': 'Piovera',
 'Nuevos Ministerios-Ríos Rosas': 'Ríos Rosas',
 'Pau de Carabanchel': 'Buenavista',
 'Pilar': 'Barrio del Pilar',
 'Tres Olivos - Valverde': 'Valverde',
 'Huertas-Cortes': 'Cortes',
 'Arroyo del Fresno': 'Mirasierra',
 'Cuzco-Castillejos': 'Castillejos',
 'Valdebernardo - Valderrivas': 'Valdebernardo',
 'Chueca-Justicia': 'Justicia',
 'Zofío': 'Zofio',
 'Lavapiés-Embajadores': 'Embajadores',
 'Buena Vista': 'Buenavista',
 'Chopera': 'La Chopera',
 'Virgen del Cortijo - Manoteras': 'Valdefuentes',
 'Costillares': 'Pinar de Chamartín',
 'Bernabéu-Hispanoamérica': 'Hispanoamérica',
 'San Andrés': 'Villaverde Alto',
 'Ensanche de Vallecas': 'Casco Histórico de Vallecas',
 'Tres Olivos': 'Valverde',
 'Valdebebas': 'Valdefuentes',
 'Virgen del Cortijo': 'Valdefuentes',
 'Puerta del Angel': 'Puerta del Ángel',
 'Rios Rosas': 'Ríos Rosas',
 'Cármenes': 'Los Cármenes',
 'Aguilas': 'Las Águilas',
 'San Cristobal': "San Cristóbal de los Ángeles",
 'Los Angeles': 'Los Ángeles',
 'Fuentelareina': 'Fuentelarreina',
 'Apostol Santiago': 'Apóstol Santiago'
 }

In [30]:
for key in item_conversion_dict:
    df['neighbourhood'] = df['neighbourhood'].replace(key, item_conversion_dict[key])

In [31]:
df['district'] = df['neighbourhood'].copy()

In [32]:
for barrio in set(df['district'].values):
    for district in madrid_districts_dict:
        if barrio in madrid_districts_dict[district]:
            df['district'] = df['district'].replace(barrio, district)

In [33]:
df_m2_price = pd.read_csv('../data/prices_m2.csv')

In [34]:
df = df[['price', 'house_type', 'house_type_2', 'has_orientation_south', 'has_orientation_north', 'has_orientation_west', 'has_orientation_east', 'rooms', 'bathrooms', 'has_terrace', 'm2', 'elevator',
       'garage', 'neighbourhood', 'district', 'built_year', 'has_pool', 'has_ac']]

In [35]:
df.head()

,price,house_type,house_type_2,has_orientation_south,has_orientation_north,has_orientation_west,has_orientation_east,rooms,bathrooms,has_terrace,m2,elevator,garage,neighbourhood,district,built_year,has_pool,has_ac
0,209995,Piso,interior,0,0,0,0,2,1,1,50.0,1,0,Las Acacias,Arganzuela,NaN,0,0
1,209995,Piso,interior,0,0,0,0,2,1,1,50.0,1,0,Embajadores,Centro,NaN,0,0
2,180000,Piso,exterior,1,0,0,0,1,1,0,50.0,1,1,Villaverde Alto,Villaverde,2007.0,1,1
3,290000,Piso,interior,0,1,0,0,1,1,0,50.0,1,0,Universidad,Centro,1880.0,0,1
4,176000,Piso,exterior,0,0,0,0,1,1,0,50.0,1,1,Rejas,San Blas-Canillejas,2006.0,1,0


In [36]:
df_add_data = pd.read_csv('../data//additional_data_cleaned_ER.csv')
df_add_data = df_add_data[(df_add_data['house_type'].str.contains('planta')) | (df_add_data['house_type'].str.contains('bajo'))]
df_add_data['house_type'] = 'Piso'

In [37]:
district_not_found = [
    ('Chamartin', 'Chamartín'),
    ('Villa-de-vallecas', 'Villa de Vallecas'),
    ('Puente-de-vallecas', 'Puente de Vallecas'),
    ('Moncloa', 'Moncloa-Aravaca'),
    ('San-blas', 'San Blas-Canillejas'),
    ('Barrio de salamanca', 'Salamanca'),
    ('Fuencarral', 'Fuencarral-El Pardo'),
    ('Ciudad lineal', 'Ciudad Lineal'),
    ('Vicalvaro', 'Vicálvaro'),
    ('Chamberi', 'Chamberí'),
    ('Tetuan', 'Tetuán')
]

In [38]:
df_add_data['district'] = df_add_data['district'].apply(lambda x: x.capitalize())

for item in district_not_found:
    df_add_data['district'] = df_add_data['district'].str.replace(item[0], item[1])

In [39]:
df_price_increase = pd.read_csv('../data/price_increase.csv')
df_add_data = df_add_data.merge(df_price_increase[['district', 'price_increase_2022']], on='district')
df_add_data['price'] = df_add_data['price']*df_add_data['price_increase_2022']
df_add_data = df_add_data.drop(columns=['price_increase_2022'])
df_add_data = df_add_data.rename(columns={'neighborhood': 'neighbourhood'})

In [40]:
def keep_true_else_false(df_init_data, list):
    for item in list:
        df_init_data[item] = df_init_data[item].apply(lambda x: 1 if x==True else 0)
    return df_init_data

In [41]:
df_init_data = pd.read_csv('../data/houses_Madrid.csv', index_col=0)
df_init_data = df_init_data[df_init_data['house_type']=='HouseType 1: Pisos']
df_init_data['house_type'] = 'Piso'

keep_true_else_false_list = ['elevator', 'garage', 'has_pool', 'has_ac', 'has_terrace', 'has_orientation_north', 'has_orientation_west', 'has_orientation_south', 'has_orientation_east']
df_init_data = keep_true_else_false(df_init_data, keep_true_else_false_list)

df_init_data['neighbourhood'] = df_init_data['neighbourhood'].apply(lambda x: x.split(' - ')[0])
df_init_data['district'] = df_init_data['district'].apply(lambda x: x.split(' - District ')[1])

for value in set(df_init_data['neighbourhood'].values):
    match = re.search(r':\s*(.*?)\s*\(', value)
    if match:
        result = match.group(1)
    else:
        match = re.search(r':\s*(.*)', value)
        result = match.group(1)
    df_init_data['neighbourhood'] = df_init_data['neighbourhood'].replace(value, result)

for value in set(df_init_data['district'].values):
    match = re.search(r':\s*(.*)', value)
    result = match.group(1)
    df_init_data['district'] = df_init_data['district'].replace(value, result)

df_init_data['district'] = df_init_data['district'].replace('Fuencarral', 'Fuencarral-El Pardo')
df_init_data['district'] = df_init_data['district'].replace('Moncloa', 'Moncloa-Aravaca')

In [42]:
for key in item_conversion_dict:
    df_init_data['neighbourhood'] = df_init_data['neighbourhood'].replace(key, item_conversion_dict[key])

In [43]:
df_init_data = df_init_data.merge(df_price_increase[['district', 'price_increase_2020']], on='district')
df_init_data['price'] = df_init_data['price']*df_init_data['price_increase_2020']
df_init_data = df_init_data.drop(columns=['price_increase_2020'])

In [44]:
df_init_data.head()

,m2,rooms,bathrooms,neighbourhood,district,price,house_type,built_year,has_ac,elevator,has_pool,has_terrace,garage,has_orientation_north,has_orientation_west,has_orientation_south,has_orientation_east
0,64.0,2,1.0,San Cristóbal de los Ángeles,Villaverde,98345.000,Piso,1960.0,1,0,0,0,0,0,1,0,0
1,70.0,3,1.0,Los Ángeles,Villaverde,150294.300,Piso,NaN,0,1,0,1,0,0,0,0,0
2,94.0,2,2.0,Villaverde Alto,Villaverde,166893.779,Piso,NaN,0,1,0,0,0,0,0,0,0
3,64.0,2,1.0,Villaverde Alto,Villaverde,127154.300,Piso,1955.0,0,1,0,0,0,0,0,1,0
4,108.0,2,2.0,Los Rosales,Villaverde,300820.000,Piso,2003.0,1,1,1,0,1,1,1,1,1


In [45]:
df_2018_1 = pd.read_json('../data/Madrid_Sale_2018_part1.json')
df_2018_2 = pd.read_json('../data/Madrid_Sale_2018_part2.json')
df_2018_3 = pd.read_json('../data/Madrid_Sale_2018_part3.json')
df_2018_4 = pd.read_json('../data/Madrid_Sale_2018_part4.json')
df_2018 = pd.concat([df_2018_1, df_2018_2, df_2018_3, df_2018_4], ignore_index=True)
df_polygons = pd.read_json('../data/Madrid_Polygons.json')

In [46]:
df_2018 = df_2018[['PRICE', 'CONSTRUCTEDAREA', 'BATHNUMBER', 'ROOMNUMBER', 'DISTANCE_TO_CITY_CENTER', 'DISTANCE_TO_METRO',  'DISTANCE_TO_CASTELLANA', 'HASNORTHORIENTATION', 'HASSOUTHORIENTATION', 'HASEASTORIENTATION', 'HASWESTORIENTATION', 'HASTERRACE', 'HASSWIMMINGPOOL', 'CADCONSTRUCTIONYEAR', 'HASLIFT', 'CONSTRUCTIONYEAR', 'HASAIRCONDITIONING', 'HASPARKINGSPACE', 'ISDUPLEX', 'LONGITUDE', 'LATITUDE']]
df_2018 = df_2018[df_2018['ISDUPLEX']==0].drop(columns=['ISDUPLEX'])
df_2018 = df_2018.rename(columns={
    'PRICE': 'price', 
    'CONSTRUCTEDAREA': 'm2',
    'BATHNUMBER': 'bathrooms',
    'ROOMNUMBER': 'rooms',
    'HASLIFT': 'elevator', 
    'CADCONSTRUCTIONYEAR': 'cad_built_year',
    'HASPARKINGSPACE': 'garage',
    'HASAIRCONDITIONING': 'has_ac',
    'CONSTRUCTIONYEAR': 'built_year', 
    'HASSWIMMINGPOOL': 'has_pool',
    'HASTERRACE': 'has_terrace',
    'DISTANCE_TO_CITY_CENTER': 'distance_city_center',
    'DISTANCE_TO_METRO': 'distance_metro',
    'DISTANCE_TO_CASTELLANA': 'distance_castellana',
    "HASNORTHORIENTATION": 'has_orientation_north',
    "HASSOUTHORIENTATION": 'has_orientation_south',
    "HASEASTORIENTATION": 'has_orientation_east',
    "HASWESTORIENTATION": 'has_orientation_west',
})

In [47]:
for index, row in df_2018.iterrows():
    if pd.isna(row['built_year']):
        df_2018.at[index, 'built_year'] = row['cad_built_year']

df_2018 = df_2018.drop(columns=['cad_built_year'])

In [48]:
df_polygons['geometry'] = df_polygons['geometry'].apply(lambda x: x['coordinates'][0][0])

geometry = []

for item in df_polygons['geometry'].values:
    new_list = []
    for sub_item in item:
        new_list.append(tuple(sub_item))
    geometry.append(new_list)

df_polygons['geometry'] = geometry

In [49]:
df_polygons.head()

,LOCATIONID,LOCATIONNAME,ZONELEVELID,geometry
0,0-EU-ES-28-07-001-079-16-002,Conde Orgaz-Piovera,8,"[(-3.6301, 40.4494), (-3.6263, 40.4495), (-3.6..."
1,0-EU-ES-28-07-001-079-16-004,Pinar del Rey,8,"[(-3.6551, 40.4668), (-3.6543, 40.4669), (-3.6..."
2,0-EU-ES-28-07-001-079-21-004,Timón,8,"[(-3.5817, 40.4797), (-3.6016, 40.4799), (-3.6..."
3,0-EU-ES-28-07-001-079-01-001,Palacio,8,"[(-3.7204, 40.4203), (-3.721, 40.42), (-3.7218..."
4,0-EU-ES-28-07-001-079-01-003,Huertas-Cortes,8,"[(-3.6965, 40.4188), (-3.6997, 40.4198), (-3.6..."


In [50]:
def check_polygon(lng, lat, df_polygons):
    point = Point(lng, lat)
    for index, row in df_polygons.iterrows():
        polygon = Polygon(row['geometry'])
        if polygon.contains(point):
            return row['LOCATIONNAME']
    return None


In [51]:
# from tqdm import tqdm

# neighbourhoods = []

# for index in tqdm(range(len(df_2018['LONGITUDE'].values)), desc="Processing points"):
#     lng = df_2018.iloc[index]['LONGITUDE']
#     lat = df_2018.iloc[index]['LATITUDE']
#     neighbourhood = check_polygon(lng, lat, df_polygons)
#     neighbourhoods.append(neighbourhood)

# df_2018['neighbourhood'] = neighbourhoods

In [52]:
# df_2018.to_csv('../data/Madrid_Sale_2018_w_neighbourhood.csv', index=False)

In [53]:
df_2018 = pd.read_csv('../data/Madrid_Sale_2018_w_neighbourhood.csv')
df_2018.shape

(92313, 20)

In [54]:
# df_2018 = df_2018.dropna()

In [55]:
df_2018.shape

(92313, 20)

In [56]:
for key in item_conversion_dict:
    df_2018['neighbourhood'] = df_2018['neighbourhood'].replace(key, item_conversion_dict[key])

In [57]:
df_2018['neighbourhood'] = df_2018['neighbourhood'].replace('El Cañaveral - Los Berrocales', 'Los Berrocales')
df_2018['neighbourhood'] = df_2018['neighbourhood'].replace('Valdebernardo - Valderribas', 'Valdebernardo')

In [58]:
districts =[]

for barrio in df_2018['neighbourhood'].values:
    found = False
    for district in madrid_districts_dict:
        if barrio in madrid_districts_dict[district]:
            districts.append(district)
            found = True
            break
    if not found:
        districts.append(None)

df_2018['district'] = districts

In [59]:
df_2018 = df_2018.drop(columns=['LONGITUDE', 'LATITUDE'])

In [60]:
df_2018.shape

(92313, 19)

In [61]:
df_2018 = df_2018.merge(df_price_increase[['district', 'price_increase_2018']], on='district')
df_2018['price'] = df_2018['price']*df_2018['price_increase_2018']
df_2018 = df_2018.drop(columns=['price_increase_2018'])

In [62]:
df = df.drop(columns='house_type_2')
df_add_data = df_add_data.drop(columns='house_type_2')

In [63]:
df = df[df['house_type']=='Piso']
df = pd.concat([df, df_add_data, df_init_data], ignore_index=True)
df = df[[column for column in df.columns if column!='house_type']]
df = pd.concat([df, df_2018], ignore_index=True)

In [64]:
df['original_index'] = df.index

df = df.merge(df_m2_price, on='district')
df = df.sort_values('original_index')
df = df.drop(columns=['original_index']).reset_index(drop=True)

In [65]:
df_airbnb = pd.read_csv('../data/airbnb_rent.csv')
df_airbnb['price'] = df_airbnb['price'].str.replace('$', '').str.replace(',', '')
df_airbnb['price'] = df_airbnb['price'].apply(lambda x: float(x))
df_airbnb = df_airbnb.rename(columns={'price': 'airbnb_price_per_night'})

/var/folders/zm/z5q67jgd0vv23h_ypgdpl9ph0000gn/T/ipykernel_99089/20744150.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_airbnb['price'] = df_airbnb['price'].str.replace('$', '').str.replace(',', '')


In [66]:
for key in item_conversion_dict:
    df_airbnb['neighbourhood'] = df_airbnb['neighbourhood'].replace(key, item_conversion_dict[key])

In [67]:
df_airbnb_mean = df_airbnb.groupby(['neighbourhood', 'rooms', 'bathrooms'])['airbnb_price_per_night'].mean().reset_index()

In [68]:
df['original_index'] = df.index
df = df.merge(df_airbnb_mean, on=['neighbourhood', 'rooms', 'bathrooms'], how='left')
df = df.sort_values('original_index')
df = df.drop(columns=['original_index']).reset_index(drop=True)

In [69]:
df.to_csv('../data/scraped_data_cleaned_Emanuela_22Jul24.csv', index=False)